<div class="markdown-google-sans">
  <h1><font size=6>Assignment 1</font></h1>

  <u>Group members:</u><br>
  - Ariel Hedvat<br>
  - Shiraz Israeli<br>
  - Yuval Bakirov<br>
  - Eitan Bakirov

<br>

In this project we are aiming to build an accurate model to predict daily bicycle rental demand using the provided bike sharing dataset. By analyzing the data and testing different modeling techniques, we will develop a robust model optimized to forecast the rental count metric on a held-out test set.<br>
The goal is to create a reliable demand prediction model for bike sharing operations.
</div>




*   כדאי להשתמש במודלים מבוססי עצים: xgboost, random forest
*   להעלות את המחברת לגיט האב ולשים את הקישור בתיבת הגשה וגם את הסי-אס-וי








<div class="markdown-google-sans">
  <h1><font size=5>Data</font></h1>
</div>

`train.csv` and `test.csv` - contain information on bike rentals, including the timestamp, seasonal indicators, holiday and working day flags, weather conditions, temperature metrics, humidity, windspeed, pollution, sunlight, traffic, and the count of bikes rented at each recorded time.<br>
Target variable to predict is "count" (Label).<br><br>


למחוק לפני הגשה:<br>

---


<u>datetime</u> - time of rental<br>
<u>season</u> - (1:winter, 2:spring, 3:summer, 4:fall)<br>
<u>holiday</u> - (Is it a bank holiday? If so: 1, else 0)<br>
<u>workingday</u> - (Is it a working day? If so: 1, else 0)<br>
<u>weather</u>
- 1: Clear, Few clouds, Partly cloudy, Partly cloudy<br>
- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist<br>
- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds<br>
- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog

<u>temp</u> - temperature <br>
<u>atemp</u> - average temperature <br>


---


<div class="markdown-google-sans">
  <h1><font size=5>Table of Contents</font></h1>

>[Import Libraries](#scrollTo=-_VBJ0JlYLKI)

>[Loading the data](#scrollTo=kXwOL606ZfUN)

>[EDA - Exploring Data Analysis](#scrollTo=P8cTl07AZgyg)

>>[Date time Features - Time Series](#scrollTo=W55ei0QSyPJ7)

>>[Features Analysis](#scrollTo=kMEpr4k3j2G6)

>>>[Categorical Features distribution](#scrollTo=jwF74W59qwf_)

>>>[Numerical Features Distribution](#scrollTo=MBRxfPjgrXVc)

>>[Label Analysis](#scrollTo=bCeLkOkokIWH)

>>[Correlation](#scrollTo=-OtGfcjyFnSI)

>>[Missing Values](#scrollTo=G15IFI5TrScB)

>>[Outliers Visualization](#scrollTo=rot7m6iisI_v)

>[Preprocessing](#scrollTo=E_OOmtAYZnm7)

>>[Add / Remove Features](#scrollTo=96lau3rN1OpT)

>>[Handling Categorial Features](#scrollTo=xtPSG9CixFck)

>>[Outliers](#scrollTo=zxs7JyHhxIoa)

>>[Data Normalizing](#scrollTo=NiFHiz6MxXGT)

>>[PCA - Dimensionality Reduction](#scrollTo=iaFEPiItxaqq)

>>[Final Preprocessing Function](#scrollTo=6TXc09TPPJe0)

>[Modelling](#scrollTo=JTkmK9sKe32b)

>>[Random forest](#scrollTo=luG0BykVe32i)

>>[Linear Regression](#scrollTo=U_CfSlB8bEqd)

>>[XGBoost](#scrollTo=nxYuD62xdzMy)

>>[Run all models](#scrollTo=Jg2jz3-de32k)

>[Evaluating on validation set](#scrollTo=qbvc7vxiZ5fC)

>>>[Overall comparison](#scrollTo=v-dUZ0sAcp9-)

>>[Feature selection & importance](#scrollTo=Y7M2ZqsX6Ofl)

>>>>[Tree based methods](#scrollTo=jl-WQpBxG49h)

>>>>[Classical regression methods](#scrollTo=eqYG1lEsG8Aa)

>>>>[Comparison](#scrollTo=aFVaV87mICFL)

>>[Add/Remove Features to Improve Results](#scrollTo=GE5x3tkL9IwM)

>>[Model improvement](#scrollTo=RGzmhm996-4c)

>[Prediction - Runing on Test](#scrollTo=dE4SF0VdaENB)

>[Output](#scrollTo=IJrSf6hnWnCJ)



#  Import Libraries

In [1]:
import pandas as pd
import numpy as np
import sklearn as skl
import seaborn as sns
import os
import scipy.stats as stats

from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
import math
from datetime import datetime, timedelta

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.ensemble import RandomForestClassifier, IsolationForest

from sklearn.decomposition import PCA

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV


from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

import xgboost as xgb

from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

# Loading the data

In [2]:
# Data Loading for Train
url = 'https://raw.githubusercontent.com/ariel-hedvat/AdvancedMLDLCourseAssignments/main/Assignment%20I/train.csv'
full_train_data = pd.read_csv(url)

train_with_labels = full_train_data.copy()
train_data = full_train_data.drop('count', axis=1).copy()
train_labels = full_train_data['count'].copy()

In [3]:
# Data Loading for Test
url_test = 'https://raw.githubusercontent.com/ariel-hedvat/AdvancedMLDLCourseAssignments/main/Assignment%20I/test.csv'
full_test_data = pd.read_csv(url_test)

test_with_labels = full_test_data.copy()
test_data = full_test_data.drop('count', axis=1).copy()
test_labels = full_test_data['count'].copy()

# **EDA - Exploring Data Analysis**

A glimpse of the data frame

In [4]:
train_data

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,pollution,sunlight,traffic
0,2011-07-11 00:00:00,3,0,1,1,28.70,32.575,65,12.9980,5.354100,28.701,0.000000
1,2012-05-18 22:00:00,2,0,1,1,22.96,26.515,52,22.0028,85.425233,22.961,0.004489
2,2011-04-01 23:00:00,2,0,1,1,12.30,15.910,61,6.0032,2.040899,12.301,0.000242
3,2012-09-16 09:00:00,3,0,0,1,23.78,27.275,60,8.9981,26.682772,23.781,0.004489
4,2011-02-01 23:00:00,1,0,1,3,8.20,9.850,93,12.9980,5.851754,8.201,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
8159,2012-01-14 02:00:00,1,0,0,1,6.56,8.335,47,11.0014,14.953355,6.561,0.004489
8160,2011-12-10 09:00:00,4,0,0,1,11.48,12.880,61,19.0012,7.977025,11.481,0.000000
8161,2011-12-18 16:00:00,4,0,0,1,11.48,13.635,48,16.9979,6.916512,11.481,0.015615
8162,2011-02-19 07:00:00,1,0,0,1,15.58,19.695,17,35.0008,0.095445,15.581,0.000242


In [5]:
train_data.shape

(8164, 12)

Dropping duplicates

In [6]:
train_data = train_data.drop_duplicates()

And again

In [7]:
train_data.shape

(8164, 12)

It can be seen that there were no identical samples in the data <br><br>Next, let's take a look at the types of features that exist:

In [8]:
train_data.dtypes

datetime       object
season          int64
holiday         int64
workingday      int64
weather         int64
temp          float64
atemp         float64
humidity        int64
windspeed     float64
pollution     float64
sunlight      float64
traffic       float64
dtype: object

Based on our knowledge of the features and the data displayed above we can conclude that: <br>

Our dataset consists 12 features and 8164 observations.
The features types :

<span style="color: orange;">`datetime`</span> is <b><u>Datetime</u></b> data type. <br>

<span style="color: #6699CC;">`temp`</span>, <span style="color: #6699CC;">`atemp`</span>, <span style="color: #6699CC;">`humidity`</span>, <span style="color: #6699CC;">`windspeed`</span>, <span style="color: #6699CC;">`pollution`</span>, <span style="color: #6699CC;">`sunlight`</span> and <span style="color: #6699CC;">`traffic`</span> - are <b><u>Numeric</u></b> data types. <br>

<span style="color: green;">`holiday`</span> and <span style="color: green;">`workingday`</span> - are <b><u>Boolean </b></u> data types.<br>

<span style="color: orange;">`season`</span> and <span style="color: orange;">`weather`</span> - are <b><u>Categorical</u></b> data types.

Hence we will update the data types of each feature:


In [9]:
numeric_features = [col for col in ['temp', 'atemp', 'humidity', 'exports', 'windspeed', 'pollution', 'sunlight', 'traffic'] if col in train_data.columns]
boolean_features = [col for col in ['holiday', 'workingday'] if col in train_data.columns]
categorical_features = [col for col in ['season', 'weather'] if col in train_data.columns]
datetime_features = ['datetime']
# features = TODO put all inside maybe

In [10]:
def change_data_types(df):

    numeric_features = [col for col in ['temp', 'atemp', 'humidity', 'exports', 'windspeed', 'pollution', 'sunlight', 'traffice'] if col in df.columns]
    boolean_features = [col for col in ['holiday', 'workingday'] if col in df.columns]
    categorical_features = [col for col in ['season', 'weather'] if col in df.columns]

    # Convert 'datetime' to datetime type
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Change numeric features to numeric data type
    df[numeric_features] = df[numeric_features].astype(float)

    # Change boolean features to boolean data type
    for col in boolean_features:
        df[col] = df[col].where(df[col].notnull(), np.nan).astype('boolean')

    # Change categorical features to categorical data type
    df[categorical_features] = df[categorical_features].astype('category')

    return df


In [11]:
train_data = change_data_types(train_data)
train_with_labels = change_data_types(train_with_labels)
test_data = change_data_types(test_data)

And after the changes:

In [12]:
train_data.dtypes

datetime      datetime64[ns]
season              category
holiday              boolean
workingday           boolean
weather             category
temp                 float64
atemp                float64
humidity             float64
windspeed            float64
pollution            float64
sunlight             float64
traffic              float64
dtype: object

In [13]:
train_with_labels.dtypes

datetime      datetime64[ns]
season              category
holiday              boolean
workingday           boolean
weather             category
temp                 float64
atemp                float64
humidity             float64
windspeed            float64
pollution            float64
sunlight             float64
traffic              float64
count                  int64
dtype: object

 ## **Date time Features - Time Series**

In this section we will check the time series attributes that the data has:

From looking at the datetime feature we see that we can seperate it to sub features: Day, Month, Year and Hour

But before separating the 'datetime' feature to sub features, we will check whether there are minutes and hours:

In [14]:
unique_times = train_data['datetime'].dt.strftime('%H:%M:%S').unique()

sorted_times = sorted(unique_times)

print("All unique times in train_data['datetime'], sorted:")
for time in sorted_times:
  print(time)

All unique times in train_data['datetime'], sorted:
00:00:00
01:00:00
02:00:00
03:00:00
04:00:00
05:00:00
06:00:00
07:00:00
08:00:00
09:00:00
10:00:00
11:00:00
12:00:00
13:00:00
14:00:00
15:00:00
16:00:00
17:00:00
18:00:00
19:00:00
20:00:00
21:00:00
22:00:00
23:00:00


In [15]:
# Count the number of unique hours
num_unique_hours = len(unique_times)

# Print the result
print(f"Number of unique hours: {num_unique_hours}")

Number of unique hours: 24


We observe that the 'datetime' column only includes hours without minutes and seconds. Therefore, we will add new features specifically for hours, days, month and year.



As you can see, bicycles can be rented at any time of the day.

# TODO - NEED TO DEL!!!

In [16]:
# def timeseries_features(df):

#   df['day'] = df['datetime'].dt.day
#   df['month'] = df['datetime'].dt.month
#   df['year'] = df['datetime'].dt.year
#   df['hour'] = df['datetime'].dt.hour

#   return df

In [17]:
def timeseries_features(df):

  df['dayInWeek'] = df['datetime'].dt.dayofweek
  df['dayInMonth'] = df['datetime'].dt.day
  df['month'] = df['datetime'].dt.month
  df['year'] = df['datetime'].dt.year
  df['hour'] = df['datetime'].dt.hour

  return df

In [18]:
def timeseries_category(df):

  # Change them to category type data
  df['dayInWeek'] = df['dayInWeek'].astype('category')
  df['dayInMonth'] = df['dayInMonth'].astype('category')
  df['month'] = df['month'].astype('category')
  df['year'] = df['year'].astype('category')
  df['hour'] = df['hour'].astype('category')

  return df

In [19]:
def timeseries_engineering(df):

  df = timeseries_features(df)
  df = timeseries_category(df)

  return df

# TODO - NEED TO DEL!!!

In [20]:
# # TODO Added : change accordingly
# def timeseries_features_v2(df):

#   df['dayInWeek'] = df['datetime'].dt.dayofweek
#   df['dayInMonth'] = df['datetime'].dt.day
#   df['month'] = df['datetime'].dt.month
#   df['year'] = df['datetime'].dt.year
#   df['hour'] = df['datetime'].dt.hour

#   return df

These new features will replace the datetime feature when training the models. For now, we will leave the datetime feature in order to perform additional analysis on it.

In [21]:
train_data = timeseries_engineering(train_data)
train_with_labels = timeseries_engineering(train_with_labels)
test_data = timeseries_engineering(test_data)

KeyError: 'day'

In [ ]:
train_with_labels.dtypes

Let's look at the sorted data and maybe draw some new conclusions:

In [ ]:
train_data

In [ ]:
start = str(train_data['datetime'].min())
end = str(train_data['datetime'].max())

print("Start: " + start[:10], "   Time: " + start[11:], "\nEnd:   " + end[:10], "   Time: " + end[11:])

Our data is all from the beginning of 2011 until December 19, 2012.

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(train_with_labels['datetime'], train_with_labels['count'])
plt.title('Time Series Plot')
plt.xlabel('Datetime')
plt.ylabel('Bike Count')
plt.show()

It can be seen that in the first half of 2011 there was a monthly increase in the amount of rented bicycles, which changed direction to decrease in the second half of 2011. The amount of rented bicycles increased again starting from 2012 until the last quarter of that year.<br>
Moreover, we can see in the diagram many intervals of identical sizes. Let's try to understand better what they say:

In [ ]:
def get_actual_and_missing_dates(df):
  unique_dates = df['datetime'].dt.date.unique()
  actual_date_strings = [str(date) for date in unique_dates]

  # Convert 'datetime' column to DatetimeIndex
  df_datetime_index = pd.DatetimeIndex(df['datetime'])

  # Get unique dates in the same format as 'expected_dates'
  expected_dates = pd.date_range(start=df_datetime_index.min(), end=df_datetime_index.max())

  # Convert expected_dates to an array of datetime.date objects
  expected_dates_array = np.array(expected_dates.date)

  expected_dates_strings = [str(date) for date in expected_dates_array]

  # Convert the date strings to sets
  expected_dates_set = set(expected_dates_strings)
  actual_dates_set = set(actual_date_strings)

  # Find the dates in expected_dates_set but not in actual_dates_set
  missing_dates = expected_dates_set - actual_dates_set

  # Convert the result back to a list
  missing_dates_list = list(missing_dates)
  actual_dates_list = list(actual_dates_set)

  # Sort the list to have the dates in ascending order
  missing_dates = sorted(missing_dates_list)
  actual_dates = sorted(actual_dates_list)

  print(f'Expected Days: {len(expected_dates_set)}')
  print(f'Actual Days: {len(actual_date_strings)}')
  print(f'Missing Days: {len(missing_dates)}')

  return actual_dates, missing_dates

In [ ]:
actual_dates_train, missing_dates_train = get_actual_and_missing_dates(train_data)

In [ ]:
actual_dates_test, missing_dates_test = get_actual_and_missing_dates(test_data)

There are 263 days in two years when bicycles were not rented.

Let's visualize the missing dates in some ways:

In [ ]:
def print_missing_dates(missing_dates):
  if len(missing_dates) > 0:
      print("Missing dates:")
      # missing_dates = sorted(missing_dates)

      month_before = '01'
      for date in missing_dates:
        month = str(date)[5:8]
        if month != month_before:
          print("-----------------------------")
        print(str(date)[:10])
        month_before = month
  else:
      print("No missing dates")

In [ ]:
print_missing_dates(missing_dates_train)

In [ ]:
print_missing_dates(missing_dates_test)

It can be seen that bicycles were not rented on the days starting from the 20th of the month until the end of the month.<br>
Next, another look:

In [ ]:
# Function to convert date strings to datetime objects
def convert_to_datetime(date_str):
    return datetime.strptime(date_str, "%Y-%m-%d")

def plot_actual_dates(missing_dates):
  # Define the date range from 1/1/2011 to 1/1/2013
  start_date = datetime(2011, 1, 1)
  end_date = datetime(2013, 1, 1)
  date_range = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]

  # Convert missing_dates to datetime objects
  missing_dates_datetime = [convert_to_datetime(date_str) for date_str in missing_dates]

  # Create a list of binary values indicating if each date is missing or not
  missing_flags = [1 if date in missing_dates_datetime else 0 for date in date_range]

  # Identify consecutive ranges of missing and not missing dates
  ranges = []
  current_range = []
  for i, flag in enumerate(missing_flags):
      if flag == 1:
          if current_range:
              ranges.append(current_range)
              current_range = []
      else:
          current_range.append(date_range[i])

  # Plotting
  plt.figure(figsize=(15, 3))  # Adjust the figure size here
  for date_range_subset in ranges:
      plt.plot(date_range_subset, [1] * len(date_range_subset), color='blue')

  plt.xlabel('Date')
  plt.ylabel('Missing Dates (1)')
  plt.title('Missing Dates Over Time')
  plt.show()

In [ ]:
plot_actual_dates(missing_dates_train)

In [ ]:
plot_actual_dates(missing_dates_test)

The data we have covers only the first 20 days of each month from January 2011 to November 2012. We don't have information for the other days of each month. Both the Train and Test datasets follow this pattern.

Because of this, using typical time series models or analyses might not be suitable. <br>The Test set shares the same timeframe but includes different hours within each day, keeping only the specific days constant.


**Insights and Comments:**

- <u>Datetime:</u>

    Each timestamp in the 'datetime' column is unique, signifying that there are no duplicated timestamp values. We have separated this column to handle the information individually, extracting details such as:

    * day
    * month
    * year
    * hour

## **Features Analysis**

### Categorical Features distribution

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_categorical_frequency(df, categorical_features):
    num_features = len(categorical_features)

    plt.figure(figsize=(14, 12))

    for i, feature in enumerate(categorical_features, 1):
        plt.subplot(2, 2, i)
        sns.countplot(x=feature, data=df)
        plt.ylabel(f'Frequency of {feature.capitalize()}', fontsize=14)
        plt.xlabel(f'{feature.capitalize()}', fontsize=12)
        plt.xticks(fontsize=10)
        plt.yticks(fontsize=10)

    plt.tight_layout()
    plt.show()

categorical_features_list = ['season', 'holiday', 'workingday', 'weather']
plot_categorical_frequency(train_data, categorical_features_list)


**From a first look, we can see that:**
Each count plot has a y-axis label indicating the frequency of the respective categorical feature.
* The 'season' feature has a similar number of instances in the data for every season.
* The majority of instances in the data have clear weather.
* Most of the days in the data are not holidays and possibly workdays.

Let's see the numbers:

In [ ]:
def categorial_repr_of_features(df, features):
  for column in features:
      categories = df[column].value_counts()
      print(f"Categories in column '{column}':")
      display(categories)
      # Noting when the categories are unique.
      if len(categories) == df.shape[0]:
          print("Each category is different.")
      print(f"Number of categories: " , len(categories), '\n')

In [ ]:
categorial_repr_of_features(train_data, ['holiday', 'workingday', 'season', 'weather'])

As we expected, there is an only 1 sample of when the weather is 4.<br>
In addition, for our research it is reasonable to estimate that in this weather the amount of bicycle rentals should decrease significantly.

**Insights and Comments:**

- <u>Season:</u>

    There are four distinct categories in the 'season' column (1, 2, 3, 4) with varying counts.
    The distribution suggests that each season is well-represented in the dataset.


- <u>Holiday and Workingday:</u>

    The 'holiday' column has two categories (0, 1) with the majority being non-holiday days (0).
    The 'workingday' column has two categories (0, 1), with a higher count for working days (1).


- <u>Weather:</u>

    The 'weather' column has four categories (1, 2, 3, 4), with the majority falling under category 1.
    Category 4 appears to have only one occurrence and might be an outlier or error.

###  **Numerical Features Distribution**

Next, we want to observe some statistics to understand what data we have: <br>
(Note that we are only looking at the numerical features and boolean features - which are represented as numbers).

In [ ]:
# Get the summary statistics of the features
print("\nSummary statistics of the features:")
train_data.describe()

Based on this information, we can make the following observations: (Generated by GPT)

Here are some conclusions and observations that can be drawn from the train_data dataframe based on the provided statistics:

- <u>Temperature and Apparent Temperature (temp & atemp):</u>

    The mean temperature is around 20.24°C, with a standard deviation of 7.80°C.
    Apparent temperature (atemp) has a similar distribution to temperature.


- <u>Humidity (in %):</u>

    The average humidity is approximately 61.84%, with a standard deviation of 19.26%.
    The minimum humidity is 0%, which might be an outlier or missing data.


- <u>Windspeed:</u>

    The average windspeed is 12.79, with a standard deviation of 8.21.
    There is a wide range of windspeed values, with a minimum of 0 and a maximum of 56.9979.


- <u>Pollution:</u>

    The pollution level has a mean of 47.15, but with a high standard deviation of 72.88.
    The pollution values range from a minimum of 0.000304 to a maximum of 754.30, suggesting potential outliers.


- <u>Sunlight:</u>

    The average sunlight is 20.25, with a standard deviation of 7.80.
    Sunlight values range from a minimum of 0.821 to a maximum of 41.001.
    Seems like it is correlated to temperature...


- <u>Traffic:</u>

    The traffic variable has a very low mean of 0.00499, with a standard deviation of 0.00632.
    The majority of the values seem to be close to zero, suggesting sparse traffic data. Its impact on the bicycle rental demand needs exploration.

Now we would like to understand how the features are structured. <br>
Are the values in each feature repeated ...? What are common values in every feature ...? Is an attribute a representative attribute with different values ...?

In [ ]:
categorial_repr_of_features(train_data, numeric_features)


**Insights and Comments:**

- <u>Temperature and Apparent Temperature (temp, atemp):</u>

    Both 'temp' and 'atemp' columns have a wide range of values, with multiple occurrences for each temperature.
    These features appear to have been discretized or rounded, resulting in multiple instances of the same temperature.


- <u>Humidity:</u>

    The 'humidity' column has 87 unique values, indicating a diverse range of humidity levels in the dataset.


- <u>Windspeed:</u>

    The 'windspeed' column has 29 unique values, with a dominant occurrence of 0.0000.
    It's possible that the windspeed values have been discretized, and 0.0000 might represent calm or very low windspeed.


- <u>Pollution:</u>

    Each value in the 'pollution' column is unique, indicating a diverse range of pollution levels.
    This feature appears to be continuous and might require further investigation for outliers.


- <u>Sunlight:</u>

    Similar to 'temp' and 'atemp', the 'sunlight' column has a variety of values with multiple occurrences for each sunlight level.


- <u>Traffic:</u>

    The 'traffic' column has four distinct values, with the majority being either 0.000000 or 0.000242.
    This variable might represent traffic intensity, and the low values suggest sparse traffic data. <br>
    Moreover, it behaves like a categorical variable with only 4 possible values - STRANGE. <br>
    Also, the numbers are very low and very close to 0.

Let's take a look on the distribution of the numerical features to draw some conclusions:

In [ ]:
plt.figure(figsize=(8, 8))
plt.rc('axes', labelsize=4)  # Adjust label font size
plt.rc('xtick', labelsize=8)  # Adjust x-axis tick font size
plt.rc('ytick', labelsize=8)  # Adjust y-axis tick font size
plt.rc('legend', fontsize=8)  # Adjust legend font size

train_data_subset = train_data.drop(columns=['datetime'])

# Plot histograms
train_data_subset.hist(ax=plt.gca())
plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()

* Seems that there might be some outliers in "windspeed" and "pollution" as stated above.

**Another look:**

In [ ]:
def create_distribution_graph(df):
    numeric_cols = df.select_dtypes(include=[float, int]).columns
    num_cols = len(numeric_cols)
    rows = int(math.sqrt(num_cols))
    cols = int(math.ceil(num_cols / rows))

    fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(35, 20))

    plot_index = 0
    for i in range(rows):
        for j in range(cols):
            if plot_index < num_cols:
                col = numeric_cols[plot_index]
                df[col].plot.density(ax=axes[i, j])
                axes[i, j].set_title(col.capitalize())
                plot_index += 1

    plt.tight_layout()
    plt.show()

create_distribution_graph(train_data)

* "Humidity" and "Pollution" show some type of normality, maybe after some tweaking a better conclusion could be drawn.

We will deal with outliers later.<br>
Hence, let's apply log transformation on these features:

In [ ]:
# Function that creates a distribution graph for all the numeric features
def create_distribution_graph(df):
    for i, col in enumerate(df.select_dtypes(include=[float, int])):
        transformed_data = np.log1p(df[col])  # Apply logarithm transformation
        transformed_data.plot.density()
        plt.title(col.capitalize() + " (Log Transformed)")
        plt.show()

create_distribution_graph(train_data)

* It can be seen that some features appear to follow a normal distribution (or something close enough) like: humidity and log of temp and log of sunlight.
* the distrdistribution of temp and sunlight look the same.

## **Label Analysis**

In [ ]:
train_with_labels

In [ ]:
plt.hist(train_with_labels['count'], bins=30, edgecolor='black')
plt.title('Distribution of Rental Count')
plt.xlabel('Rental Count')
plt.ylabel('Frequency')
plt.show()

In this chart we can see how many times there was any amount of bike rentals. For example, there were more than 1750 times (hours) where there were 0 bike rentals.

**Box Plot for Target Variable vs. Categorical and Boolean Features to explore how the target variable varies across different categories:**

Let's see some information about the count of the bike rentals there were in each category:

In [ ]:
# Box Plot for Target Variable vs. Categorical and Boolean Features to explore how the target variable varies across different categories.
categorical_features = ['season', 'holiday', 'workingday', 'weather']
fig, axes = plt.subplots(nrows=1, ncols=len(categorical_features), figsize=(16, 6))
for i, feature in enumerate(categorical_features):
    sns.boxplot(x=feature, y='count', data=train_with_labels, ax=axes[i])
    axes[i].set_title(f'Boxplot of Count by {feature}')
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel('Count')

plt.tight_layout()
plt.show()

We can understand things about the median and the percentiles of the amount of bike rentals in each category.

Let's analyze the distribution of bike rentals across distinct categories within specific categorical features ('season,' 'holiday,' 'workingday,' and 'weather').

In [ ]:
# Calculate the total count of bike rentals
total_rentals = train_with_labels['count'].sum()

# List of categorical features
categorical_features = ['season', 'holiday', 'workingday', 'weather']

# Create subplots with a single row and multiple columns
fig, axes = plt.subplots(nrows=1, ncols=len(categorical_features), figsize=(15, 5))

# Iterate over each categorical feature
for i, feature in enumerate(categorical_features):
    # Calculate the percentage of bike rentals for each category
    percentages = (train_with_labels.groupby(feature)['count'].sum() / total_rentals) * 100

    # Create a bar plot for each categorical feature
    sns.barplot(x=percentages.index, y=percentages, ax=axes[i])
    axes[i].set_title(f'Percentage of Bike Rentals by {feature.capitalize()}')
    axes[i].set_ylabel('Percentage')

# Adjust the layout
plt.tight_layout()
plt.show()



These charts calculate the percentage of bike rentals for each category. <br>For example, only 15 percent of the bike rentals happened in season 1.<br>
An overwhelming majority of the bike rentals happened on weekdays and not on holidays.<br>

It is difficult to draw conclusions from these data due to the imbalance in the data. We will have difficulty understanding the behavior of the bicycle rentals in each of the categories due to the imbalance in the data and we would like to normalize this data in order to better study the nature of the bicycle rentals. For example, there are many more weekdays than holidays - the data will be skewed so that naturally more bikes will be rented on weekdays because there are more weekdays in the data.


In [ ]:
total_bike_rentals = train_with_labels['count'].sum()

print(f"Total number of bike rentals in the dataset: {total_bike_rentals}")

In [ ]:
# List of categorical features
categorical_features = ['season', 'weather', 'holiday', 'workingday']

# Loop through each categorical feature
for feature in categorical_features:
    # Calculate the sum of rental bikes for each category
    total_count_by_category = train_with_labels.groupby(feature)['count'].sum()

    # Print the results
    print(f"Total rental bikes by {feature}:")
    print(total_count_by_category)
    print("\n" + "-"*30 + "\n")

To address the imbalance in the data and provide a fair comparison across categories, we toke the sum of bike rentals and divided it by the amount of each category instance. then we got the average count of rentals per unit in every categorial feature. This way, we normalized the rental counts by the frequency of each category.


This code calculates the mean number of rental bikes for each category within the specified categorical features:

In [ ]:
# List of categorical features
categorical_features = ['season', 'weather', 'holiday', 'workingday', 'day']

# Loop through each categorical feature
for feature in categorical_features:
    # Calculate the mean number of rental bikes for each category
    mean_count_by_category = train_with_labels.groupby(feature)['count'].mean()

    # Print the results
    print(f"Mean rental bikes by {feature}:")
    print(mean_count_by_category)
    print("\n" + "-"*30 + "\n")

This code creates a set of pie charts to visualize the mean number of rental bikes for each category within the specified categorical features:

In [ ]:
# List of categorical features
categorical_features = ['season', 'weather', 'holiday', 'workingday', 'day']

# Set up subplots
fig, axes = plt.subplots(1, len(categorical_features), figsize=(20, 6))

# Loop through each categorical feature
for i, feature in enumerate(categorical_features):
    # Calculate the mean number of rental bikes for each category
    mean_count_by_category = train_with_labels.groupby(feature)['count'].mean()

    # Plot the means in a pie chart
    axes[i].pie(mean_count_by_category, labels=mean_count_by_category.index, autopct='%1.1f%%', startangle=90)
    axes[i].set_title(f"Mean Rental Bikes by {feature}")

# Adjust layout
plt.tight_layout()
plt.show()

**From the visualization above, we get the following insights:**

* **Season:**
Rentals tend to be lower in Season 1 (Spring) compared to the other seasons.
Season 3 (Fall) has the highest mean rental count.

* **Weather:**
Weather conditions 1 (Clear, Few clouds) have the highest mean rental count.
Weather condition 3 (Light Snow, Light Rain, Thunderstorm) has the lowest mean rental count.


* **Holiday:**
There is a slight decrease in mean rental count on holidays compared to non-holidays, but the difference is not very significant.

* **Workingday:**
Mean rental counts are slightly higher on working days compared to non-working days.<br><br>


 The insights offer an overview of average rental counts across various categorical features. It seems that factors such as season and weather exert more noticeable effects on rental counts compared to holiday or working day status. Overall, although there are some variations in mean rental counts across different feature categories, **the differences may not be highly significant.**

In [ ]:
#This code calculates and plots the percentage of bike rentals by day of the week.

total_counts_by_day = train_with_labels.groupby(train_with_labels['datetime'].dt.dayofweek)['count'].sum()
percentage_rentals_by_day = (total_counts_by_day / total_counts_by_day.sum()) * 100

days_labels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

plt.figure(figsize=(8, 8))
plt.pie(percentage_rentals_by_day, labels=days_labels, autopct='%1.1f%%', startangle=90, colors=plt.cm.Set3.colors)
plt.title('Percentage of Bike Rentals by Day of the Week')
plt.show()


From the visualization above of the bike rental percentages for each day, we observe a consistent pattern throughout the week. The rental percentage remains relatively stable at between 14.1% and 14.9% for each day, except for Sunday, where it slightly decreases to 13.4%. This slight deviation aligns with expectations as Sunday is typically a non-working day. But we consider the variation as minimal, so we could say it is a fairly uniform distribution of bike rentals across the days of the week, with Sunday being only marginally lower but still in close proximity to the 14% mark observed on other days.

## Correlation

**trying to learn about the relationships between numerical features and the target variable using a correlation matrix**

In [ ]:
plt.figure()
correlation_matrix = train_with_labels.corr()
correlation_with_label = correlation_matrix['count']

# Remove the label feature from the correlation
correlation_with_label = correlation_with_label.drop('count')

correlation_with_label.plot(kind='bar', color='skyblue')
plt.title('Correlation with Count Label')
plt.xlabel('Features')
plt.ylabel('Correlation')
plt.xticks(rotation=45)
plt.show()

**Insights from the presentation of the correlation:**

*  The correlation tables provide insights into the relationships between variables in a dataset, especially focusing on how variables are associated with each other and, in the case of the second table, their correlation with a specific target variable ('count').
*   'polution' feature is highly correlated with the label (0.6), that is, it can be concluded that as air pollution increases, there are more bike rentals and vice versa.
*   'traffic' has a very low correlation with both the target variable ('count') and other features.


Let's make a full correlation matrix:

In [ ]:
def corr_matrix(df):
  corr_matrix = df.corr().round(1)  # Round the correlation values to 1 decimal place
  plt.figure(figsize=(12, 10))
  sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', linewidths=.5)  # Set fmt='.1f' to display 1 decimal place
  plt.title('Correlation Heatmap')
  plt.show()

corr_matrix(train_with_labels)

This function creates a visual representation of the correlation matrix using a color-coded heatmap.<br>
It displays a grid of squares, where each square represents the correlation between two features.

Here are some general observations we can make from the heatmap:

<b> Strong positive correlation:</b> If two features have a high positive correlation, it suggests that as one feature increases, the other feature tends to increase as well. Conversely, if two features have a strong negative correlation (close to -1), it means that as one feature increases, the other feature tends to decrease.

<b>Weak or no correlation:</b> If the correlation coefficient is close to 0, it indicates a weak or no linear relationship between the features. This means that changes in one feature do not necessarily correspond to changes in the other feature.

<b>Redundant or highly correlated features:</b> High correlation values between pairs of features might indicate that these features provide similar information, in our case: sunlight & temp & atemp. Sunlight, temperature, and average temperature (atemp) show multicollinearity because they're closely related in weather patterns. Clear and sunny days typically come with both higher temperatures, making these variables correlated and potentially redundant in predicting outcomes.


**Insights from the correlation matrix:**

* As we saw before, the label "count" is highly correlated with the feature "pollution" (0.6).
* Correlation of 0.4 (not too low) with: temp, atemp, sunlight with count.
* We will observe that there are also numerous instances of zeros, indicating an absolute absence of correlation.
* We can see that there is high correlation (1) between the features "temp", "atemp" and "sunlight".

**VIF:**

Let's take a look in general. <br>
In this part we will use a new method which was not learnt in class- the VIF method.
We want primarily observe whether a feature has a high correlation with other features based on its Variance Inflation Factor (VIF) value.<br>
A high VIF value indicates multicollinearity, which suggests a strong correlation between the feature and other features in the dataset.

In [ ]:
# Select numerical features (excluding the categorical features)
numerical_features = train_data.select_dtypes(include=['float64', 'bool'])

# Convert boolean features to numeric (0 and 1)
boolean_features = numerical_features.select_dtypes(include='bool')
boolean_features = numerical_features.dropna()
numerical_features[boolean_features.columns] = boolean_features.astype(int)

# Remove any rows with null values in the selected features
numerical_features = numerical_features.dropna()

# Calculate VIF for each feature
vif = pd.DataFrame()
vif["Variable"] = numerical_features.columns
vif["VIF"] = [variance_inflation_factor(numerical_features.values.astype(float), i) for i in range(numerical_features.shape[1])]

print("VIF for all features except category features:")
print(vif)

To calculate the VIF, we regress each predictor variable against all the other predictor variables in the model. The VIF for each variable is then computed as the ratio of the variance of the estimated regression coefficient to the variance of the coefficient if that variable was uncorrelated with the other predictors.

Features with VIF values close to 1 (around or below 1) indicate low multicollinearity. These features are relatively independent of each other when predicting the target variable.<br>
Examples: None.<br>

Features with VIF values between 1 and 5 are generally considered to have moderate multicollinearity. Although there might be some correlation, it is not severe.<br>
Examples: "holiday", "workingday", "windspeed", "pollution".<br>

Features with VIF values above 5 suggest the presence of multicollinearity. These features have a strong correlation with other features in the dataset and may negatively impact the model's performance.<br>
Examples: "temp", "atemp", "humidity", "sunlight".
<br><br>
Based on this information, we can consider the VIF values to identify potential issues related to multicollinearity.<br>
 High VIF values indicate that certain features are highly correlated with others, which can affect the model's interpretability and stability.<br>
In such cases, we may consider removing or transforming the highly correlated features to mitigate multicollinearity and improve the model's performance.

**The feature "traffic" again doing some problems outputting NAN. We will consider removing it because it is behaving abnormally.**




In conclusion, the VIF index helped us to understand in general whether there is a correlation for features.
The correlation matrix looked linearly at whether there was a correlation between each 2 features.
In both of these indices we saw that the "temp", "atemp" and "sunlight" features has a very high correlation.
**All in all, later, we will maybe consider dropping some of them to improve the model.**

## Missing Values

We will check the amount of NULL valuse (We can see that there are no null values):

In [ ]:
train_data.isnull().sum().sort_values(ascending = False)

There is no need to handle missing values in the pre-process due to no null values as we can see above.


## Outliers Visualization

In order to identify potential outliers,
lets visualize, as a start, the boxplots of each non-categorial feature:

In [ ]:
train_data.plot(kind="box",subplots=True,layout=(6,3),figsize=(15,30));

In general, from boxplots we can see outliers and the distribution of features.
We can see that 'temp', 'atemp', 'humidity', 'windspeed' and 'sunlight' are normally distributed.

Also, seems like 'pollution' has a lot of outliers and 'windspeed' has some less outliers.

Log transformation compresses the range of the variable. In other words, it brings large values closer together and spreads out small values. This compression can help reduce the influence of extreme values.

Also, log transformation can make the distribution of a variable more symmetric. In many cases, taking the logarithm can pull the outliers back towards the center.

So, before getting to final conclusions, lets see the log-boxplots of these features:


In [ ]:
# Selecting float-type features
float_features = train_data.select_dtypes(include=[np.float64, np.float32, np.int64])

# Applying logarithmic transformation
log_train_data = float_features.apply(np.log1p)

# Creating a copy of the original DataFrame
transformed_train_data = train_data.copy()

# Renaming columns with 'log-' prefix
log_feature_names = ['log-' + col for col in log_train_data.columns]
log_train_data.columns = log_feature_names

# Plotting boxplots with logarithmic values
log_train_data.plot(kind="box", subplots=True, layout=(6, 3), figsize=(20, 40))
plt.show()

Based on the earlier distribution visualisations and the box plots presented above, it appears that:
* **'temp' and 'atemp'** - are normaly distributed.
* **'humidity'** - is normaly distributed. Also, 0% and 100% are a bit extreme for values so we need to make sure we clear those out.
* **log of 'pollution'** - is close enough to normally distributed.
* **'sunlight'** - is normaly distributed.
* **'traffic'** - strenghtening our hypothesis that this feature should be removed.

Final check for normality just to be sure:

In [ ]:
# Determine the number of rows and columns for the grid
num_rows = 3  # Number of rows in the grid
num_cols = 2  # Number of columns in the grid

# Select specific features from each DataFrame
selected_features_train = train_data[['temp', 'atemp', 'humidity', 'sunlight']]
selected_features_log_train = log_train_data[['log-pollution']]

# Combine the normaly distributed data to one DataFrame
ND_data = pd.concat([selected_features_train, selected_features_log_train], axis=1)

# Create the grid of subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 10))

# Flatten the axes array for easier indexing
axes = axes.flatten()

# Iterate over each column in ND_data
for i, column in enumerate(ND_data.columns):
    # Create Q-Q plot
    stats.probplot(ND_data[column], dist="norm", plot=axes[i])

    # Set plot title
    axes[i].set_title(f"Q-Q Plot for {column}")

# Adjust the spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()


Seems like indeed our findings were true:

* **'temp' and 'atemp'** - are normally distributed.
* **'humidity'** - is normally distributed. Also, 0% and 100% are a bit extreme for values so we need to make sure we clear those out.
* **'sunlight'** - is normally distributed.
* **log of 'pollution'** - is close enough to normally distributed.

In order to find outliers we categorized the features to normaly distributed and not normaly disbributed, By finding the normaly distributed features, we will be able to implement the IQR method for outliers and remove them accordingly later.


In the functions below we will plot the outliers according to the IQR method with several plots and draw conclusions afterwards:

In [ ]:
def identify_bounds(df, feature, lower_percentile = 0.25, upper_percentile = 0.75, threshold=1.5):
    # Calculate the specified percentiles
    p1 = df[feature].quantile(lower_percentile)
    p2 = df[feature].quantile(upper_percentile)
    spread = p2 - p1

    # Define the upper and lower bounds
    lower_bound = p1 - threshold * spread
    upper_bound = p2 + threshold * spread

    # print(f"For {feature} the threshold given {threshold}, Lower Bound: '{lower_bound}', Upper Bound: '{upper_bound}'.")
    return lower_bound, upper_bound

In [ ]:
def visualize_outliers_histogram(df, feature, ax, lower_bound, upper_bound, fontsize=10):
    # Plot histogram of the feature
    ax.hist(df[feature], bins=20)
    ax.set_title(f"Histogram of {feature}")
    ax.set_xlabel(feature, fontsize=10)
    ax.set_ylabel("Frequency", fontsize=10)

    # Mark the outliers on the plot
    outliers = df[(df[feature] < lower_bound) | (df[feature] > upper_bound)]
    ax.scatter(outliers[feature], np.zeros_like(outliers[feature]), color='red', marker='x', label='Outliers')

    ax.legend()

def visualize_outliers_scatter(df, feature, ax, lower_bound, upper_bound, fontsize=10):
    # Identify the outliers
    outliers = df[(df[feature] < lower_bound) | (df[feature] > upper_bound)]

    # Visualize the outliers
    ax.scatter(df['count'], df[feature], color='blue', label='Data')
    ax.scatter(outliers['count'], outliers[feature], color='red', label='Outliers')
    ax.set_xlabel('Bike Rental Count', fontsize=10)
    ax.set_ylabel(feature, fontsize=10)
    ax.set_title(f'Outliers in {feature}')
    ax.legend()

def visualize_outliers_boxplot(df, feature, ax, lower_bound, upper_bound, fontsize=10):
    # Create a boxplot of the feature
    sns.boxplot(data=df, y=feature, ax=ax, width=0.5)
    ax.set_xlabel(f'{feature}', fontsize=10)
    ax.set_ylabel('Values', fontsize=10)
    ax.set_title(f'Boxplot of {feature}')


def plot_grid_outliers(df, features, plotting_funcs):
    num_cols = len(features)
    num_funcs = len(plotting_funcs)
    rows = num_cols
    cols = num_funcs

    fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(18, 5 * rows))

    axes = axes.reshape(-1)  # Reshape axes to a 1D array

    for i, feature in enumerate(features):
        lower_bound, upper_bound = identify_bounds(df, feature)

        for j, plot_func in enumerate(plotting_funcs):

            ax = axes[i * num_funcs + j]  # Get the correct axis
            plot_func(df, feature, ax, lower_bound, upper_bound)

    fig.suptitle('Outlier view for the Normally Distributed Features\n\n', fontsize=16, wrap=True)
    # plt.figtext(0.5, 0.95, 'Outlier view for the Normally Distributed Features', ha='center', fontsize=16)
    plt.subplots_adjust(top=0.9)  # Adjust the top spacing
    plt.tight_layout()
    plt.show()


In [ ]:
def plot_normal_dist_outliers(IQR_data, IQR_features = ['temp', 'humidity', 'sunlight', 'log-pollution']):

    plotting_funcs = [visualize_outliers_histogram, visualize_outliers_scatter, visualize_outliers_boxplot]
    plot_grid_outliers(IQR_data, IQR_features, plotting_funcs)

In each row we plot one of the normally distributed features and mark the outliers.

In [ ]:
ND_data = pd.concat([ND_data, train_labels], axis=1)
plot_normal_dist_outliers(ND_data)

Looks like there are not many outliers.<br>
Using the IQR method we will remove all the outliers which fall below Q1 – 1.5 IQR or above Q3 + 1.5 IQR.


In the preprocessing section we will handle these as decided.

Not forgetting about the other non-normally distributed, we will handle their outliers using Isolation Forest algorithm in the preprocessing section as well.

# Preprocessing

* We will note that:

for tree models there is no need to deal with categorical features or data normalization because the tree knows how to deal with it, but it is necessary for a linear regression model and therefore we will perform this procedure for it.

Here we would like to make sure that we are making a correct copy of the data that we will go over and make the changes in the pre-processing stage:

In [ ]:
full_train_data_copy = full_train_data.copy()
Y_df = full_train_data_copy['count']
X_df = full_train_data_copy.drop('count', axis=1)

# Train split to train and validation with fixed random state (42) to ensure reproducibility
X_train, X_val, Y_train, Y_val = train_test_split(X_df, Y_df, test_size=0.2, random_state = 42, shuffle = True)

#We will copy the data for trees and data for linear regression models
X_train_tree = X_train.copy()
Y_train_tree = Y_train.copy()
X_train_lr = X_train.copy()
Y_train_lr = Y_train.copy()

X_val_tree = X_val.copy()
Y_val_tree = Y_val.copy()
X_val_lr = X_val.copy()
Y_val_lr = Y_val.copy()

# Save copies in order to have a clean data set to use in the final preprocessing function
# (when pipelining all the functions from start to end)
X_train_tree_original = X_train.copy()
Y_train_tree_original = Y_train.copy()
X_val_tree_original = X_val.copy()
Y_val_tree_original = Y_val.copy()

X_train_lr_original = X_train.copy()
Y_train_lr_original = Y_train.copy()
X_val_lr_original = X_val.copy()
Y_val_lr_original = Y_val.copy()

## Add / Remove Features

Here we would like to add or delete features according to the conclusions from the EDA:

* **Insights from the correlation matrix -** We believed it would be more meaningful to keep temperature in the data and remove atemp and sunlight to enhance the accuracy in capturing the prevailing weather conditions at that specific time of the day. From a temperature feature (temp) we can  learn a lot about the average temperature feature (atemp) and the sunlight feature (sunlight).

* **'traffic' -** We would like to remove it from the data because it has a very low correlation with the label. Additionally, it contains unusual numbers whose meaning is difficult to explain, and the overall significance of this feature is unclear.

* **'datetime' -** We have split it into its constituent parts, providing all the necessary information for training the model. The datetime type holds no significance in this project; therefore, we opted to break it down into these categorical features: day, month, year, and hour, and remove 'datetime' fom the data.

* **'pollution'** - Log transformation brings large values closer together and spreads out small values. This compression can help reduce the influence of outliers. As we saw before in the boxplots, the 'pollution' feature has a lot of outliers, while the boxplot of the log transformation of this feature does not, so the transformation makes it more suitable for enhancing our model training. Consequently, we prefer to exclude 'pollution' and incorporate the logarithm of 'pollution' in our model.

TODO - 'Windspeed' feature to log ?!

In [ ]:
def adding_new_features(df):

  # Adding log of pollution
  df['log_pollution'] = np.log1p(df['pollution'])

  # Adding day, month, year and hour features (function from EDA)
  df = timeseries_engineering(df)

  return df

In [ ]:
def remove_features(df):
  # Remove 'atemp' and 'sunlight' variables to address multicollinearity
  multicollinearity_features = ['atemp', 'sunlight']
  df.drop(multicollinearity_features, axis=1, inplace=True)

  #remove traffic
  df.drop(['traffic'], axis=1, inplace=True)

  #remove datetime
  df.drop(['datetime'], axis=1, inplace=True)

  #remove pollution
  df.drop(['pollution'], axis=1, inplace=True)

  return df

In [ ]:
def feature_engineering_for_tree(df):
  df['datetime'] = pd.to_datetime(df['datetime'])

  df = adding_new_features(df)

  df = timeseries_features(df)

  df = remove_features(df)
  return df

In [ ]:
X_train_tree = feature_engineering_for_tree(X_train_tree)

In [ ]:
X_val_tree = feature_engineering_for_tree(X_val_tree)

This is relevant to linear regression (in order to deal with categorial features):

* Adding and removing features according to our logic:
Now we will deal with the features of datetime - If we will keep it as this, we will get too much features.

Dividing the year into quarters:

* **'month'** - Since there are many months (12 months), we would prefer to divide the year into quarters and reduce the number of features. We will add 'time_in_year' feature:
  * 1 - 1/4 of the year (1-3 months)
  * 2 - 2/4 of the year (4-6 months)
  * 3 - 3/4 of the year (7-9 months)
  * 4 - 4/4 of the year (10-12 months)

In [ ]:
def add_time_in_year_feature(month):
    if 1 <= month <= 3:
        return 1  # 1st quarter
    elif 4 <= month <= 6:
        return 2  # 2nd quarter
    elif 7 <= month <= 9:
        return 3  # 3rd quarter
    else:
        return 4  # 4th quarter

Dividing the month into thirds:

* **'day'** - Since there are many days (31 days), we would prefer to divide the month into thirds and reduce the number of features. We will add 'time_in_month' feature:
  * 1 - Begging of the month (1-10 days)
  * 2 - Middle of the month (11-21 days)
  * 3 - End of the month (22-31 days)

In [ ]:
def add_time_in_month_feature(day):
    if 1 <= day <= 10:
        return 1  # Beginning of the month
    elif 11 <= day <= 21:
        return 2  # Middle of the month
    else:
        return 3  # End of the month

Dividing the day into times:

* **'hour' -** We realized that there are too many hours (24 hours). We would prefer to categorize the data into time ranges: morning, afternoon, and evening. This is done to reduce the number of dimensions.
We will add 'time_in_day' feature:
  * 1 - Morning
  * 2 - Afternoon
  * 3 - Evening

In [ ]:
def add_time_in_day_feature(hour):
  if 4 <= hour < 12:
    return 1  # Morning
  elif 12 <= hour < 20:
    return 2  # Afternoon
  else:
    return 3  # Evening

Apply functions and remove: 'month' , 'day' and 'hour':

In [ ]:
def dealing_with_times(df):
  # Adding time in year feature
  df['time_in_year'] = df['month'].apply(add_time_in_year_feature)
  # Adding time in month feature
  df['time_in_month'] = df['dayInMonth'].apply(add_time_in_month_feature)
  # Adding time in day feature
  df['time_in_day'] = df['hour'].apply(add_time_in_day_feature)
  # Removing 'month', 'day' and 'hour'
  df.drop(['month', 'dayInMonth', 'hour'], axis=1, inplace=True)
  return df

# TODO - NEED TO DEL!!!

In [ ]:
# # TODO Added : change accordingly
# def dealing_with_times_v2(df):

#   # Adding time in year feature
#   df['time_in_year'] = df['month'].apply(add_time_in_year_feature)
#   # Adding time in month feature
#   df['time_in_month'] = df['dayInMonth'].apply(add_time_in_month_feature)
#   # Adding time in day feature
#   df['time_in_day'] = df['hour'].apply(add_time_in_day_feature)

#   return df

In [ ]:
# def feature_engineering(df):
#   df['datetime'] = pd.to_datetime(df['datetime'])
#   df = adding_new_features(df)
#   df = dealing_with_times(df)
#   df = remove_features(df)
#   return df

In [ ]:
# # TODO Added : change accordingly
# def feature_engineering_v2(df):
#   df['datetime'] = pd.to_datetime(df['datetime'])

#   df = adding_new_features(df)

#   df = timeseries_features_v2(df)

#   df = dealing_with_times_v2(df)

#   df = remove_features(df)
#   return df

In [ ]:
def feature_engineering_for_linear(df):
  df['datetime'] = pd.to_datetime(df['datetime'])

  df = adding_new_features(df)

  df = dealing_with_times(df)

  df = timeseries_features(df)

  df = remove_features(df)
  return df

In [ ]:
X_train_lr = feature_engineering_for_linear(X_train_lr)
X_val_lr = feature_engineering_for_linear(X_val_lr)

In [ ]:
X_train_lr

In [ ]:
X_val_lr

## Handling Categorial Features


From the stages before we know which features are categorial and which are not:

In [ ]:
categorial_features = ['year', 'time_in_month', 'time_in_day', 'time_in_year', 'season', 'weather', 'dayInWeek']
non_categorial_features = ['holiday', 'workingday', 'temp', 'humidity', 'windspeed', 'log_pollution']

Here we are using the OneHotEncoder to encode the specified categorical features, creating dummy variables for each category.

In [ ]:
def OHE_categorial_features(df, categorial_features):
  # Create an instance of OneHotEncoder
  encoder = OneHotEncoder(sparse=False)

  # Fit and transform the selected columns
  encoded_columns = encoder.fit_transform(df[categorial_features])

  # Create custom feature names for the encoded columns
  feature_names = []
  for i, column in enumerate(categorial_features):
      categories = encoder.categories_[i]  # Use encoder.categories_ to get categories
      for feature in categories:
          feature_names.append(f'{column}_{feature}')

  # Create a DataFrame with the encoded columns and custom feature names
  encoded_data = pd.DataFrame(encoded_columns, columns=feature_names)
  # print(encoded_data.shape)

  data_reset = df.reset_index(drop=True)
  encoded_data_reset = encoded_data.reset_index(drop=True)

  # Concatenate the encoded columns with the original DataFrame
  data_encoded = pd.concat([data_reset, encoded_data_reset], axis=1)

  df = data_encoded

  df.drop(columns=categorial_features, inplace=True)
  return df

In [ ]:
X_train_lr = OHE_categorial_features(X_train, categorial_features)
X_val_lr = OHE_categorial_features(X_val, categorial_features)

In [ ]:
X_train_lr.columns

In [ ]:
X_val_lr.columns

Logically: after splitting into times by year, month, day and hour, then:
* Day - there are 7 days a week
* Month - there are 12 months in a year
* Hour - there are 24 round hours in a day
* Year - according to the data (the years that appear in the training data and additional years that may appear)

Also:
* Season and Weather - should be splitted into 4 categories.

That's why we have to make sure that all the dummy categories appear in every data set we will work with.

In [ ]:
def ensure_all_categories(df, expected_categories):
    # Iterate through each categorical feature
    for feature, categories in expected_categories.items():
        # Check if the feature is in the DataFrame columns
        df_columns_list = list(df.columns)
        missing_categories = [cat for cat in categories if cat not in df_columns_list]

        # Add missing categories to the DataFrame
        if missing_categories:
            df = pd.concat([df, pd.DataFrame(0, index=df.index, columns=missing_categories)], axis=1)

        # Fill added categories with 0s
        df[categories].fillna(0, inplace=True)

    return df

In [ ]:
def build_expected_categories():
    expected_categories = {}

    # Define categories for 'day'
    expected_categories['time_in_year'] = [f'time_in_year_{i}' for i in range(1, 5)]

    # Define categories for 'month'
    expected_categories['time_in_month'] = [f'time_in_month_{i}' for i in range(1, 4)]

    # Define categories for 'hour'
    expected_categories['time_in_day'] = [f'time_in_day_{i}' for i in range(1, 4)]

    # Define categories for 'year'
    expected_categories['year'] = ['year_2011', 'year_2012', 'year_other']

    # Define categories for 'season'
    expected_categories['season'] = [f'season_{i}' for i in range(1, 5)]

    # Define categories for 'weather'
    expected_categories['weather'] = [f'weather_{i}' for i in range(1, 5)]

    return expected_categories

In [ ]:
expected_categories = build_expected_categories()
X_train_lr = ensure_all_categories(X_train_lr, expected_categories)
X_val_lr = ensure_all_categories(X_val_lr, expected_categories)

In [ ]:
X_train_lr.columns

In [ ]:
X_val_lr.columns

Now we need to make sure that the order of the features in every df is the same - we will match our dfs according to the list of the categories we made:

In [ ]:
fixed_categorial_features = []
for values_list in expected_categories.values():
    fixed_categorial_features.extend(values_list)

In [ ]:
X_train_lr = X_train_lr[non_categorial_features+fixed_categorial_features]
X_val_lr = X_val_lr[non_categorial_features+fixed_categorial_features]

In [ ]:
X_train_lr.columns

In [ ]:
X_val_lr.columns

Let's create a wrapping function that calls all the functions at once:

In [ ]:
def handle_categorial_data(df):
  categorial_features = ['year', 'time_in_month', 'time_in_day', 'time_in_year', 'season', 'weather', 'dayInWeek']
  non_categorial_features = ['holiday', 'workingday', 'temp', 'humidity', 'windspeed', 'log_pollution']

  df = OHE_categorial_features(df, categorial_features)

  expected_categories = build_expected_categories()

  df = ensure_all_categories(df, expected_categories)

  fixed_categorial_features = []
  for values_list in expected_categories.values():
    fixed_categorial_features.extend(values_list)

  df = df[non_categorial_features+fixed_categorial_features]

  return df

##  Outliers


As discussed above, we saw that there are several features that are normally distributed and some that do not.
Therfore, we will handle these as follows:

- There were a lot of outliers in 'pollution' so we applied log transformation on it and concluded that this feature is normally distributed.
- On the <u>Normally distributed</u> which are 'temp', 'humidity' and 'log_pollution' we will use the IQR method to identify them and replace them with the bound they are close to (Upper or Lower).

- On the <u>Non-normally distributed feature</u> which is 'windspeed' we will apply an Isolation Forest algorithm by learning on this feature and creating a decision whether a record is labeled as outlier or not. If it does, we will remove it.


<u>Normally Distributed Features:</u>

Applying the IQR Method on the train data - identifying the bounds of the normally distributed features and replacing them with the bounds accordingly:

In [ ]:
ND_features = ['temp', 'humidity', 'log_pollution']
NND_features = ['windspeed']

In [ ]:
def handle_ND_outliers_train(data, ND_features):

    # log_features = ['pollution']
    # data = apply_log(data, log_features)

    # normal_features = ['temp', 'humidity', 'log_pollution']
    bounds = {}

    for feature in ND_features:
        lower_bound, upper_bound = identify_bounds(data, feature)

        data[feature] = np.where(data[feature] < lower_bound, lower_bound, data[feature])
        data[feature] = np.where(data[feature] > upper_bound, upper_bound, data[feature])

        bounds[feature] = (lower_bound, upper_bound)

    return data, bounds

In [ ]:
def handle_ND_outliers_test(data, feature_bounds):

    # log_features = ['pollution']
    # data = apply_log(data, log_features)

    for feature, (lower_bound, upper_bound) in feature_bounds.items():
        data[feature] = np.where(data[feature] < lower_bound, lower_bound, data[feature])
        data[feature] = np.where(data[feature] > upper_bound, upper_bound, data[feature])

    return data

<u>Non-Normally Distributed Features:</u><br>


For this task, we decided to use Isolation Forest algorithm. <br>
This algorithm trains and learns the bounds of each feature and then can be applied on every chosen dataset.
Since we want the train data to have the same charactaristics as the test model, we preffered to classify small amount of data as outliers.
Also, by doing so, we reduce the chances to overfitting, because the model trains on data that is pretty simillar to the tested one.
Therefore we chose the threshold and contamination hyperparameter to be both 0.01. TODO - maybe change the hyperparameter

Create the function to handle the non-normally distributed features in the training data:

In [ ]:
def handle_NND_outliers_train(data, labels, threshold, contamination, NND_features):

  # Initialize the Isolation Forest model
  model = IsolationForest(contamination=contamination)

  # NND_features = ['windspeed']
  data_selected = data[NND_features].copy()

  model.fit(data_selected)

  outlier_scores = model.decision_function(data_selected)
  outlier_indices = np.where(outlier_scores < threshold)[0]

  data.reset_index(drop=True, inplace=True)
  labels.reset_index(drop=True, inplace=True)

  # Remove the outlier indices from the data
  data = data.drop(outlier_indices)
  labels = labels.drop(outlier_indices)

  return data, labels

Binding both of the decisions together:

In [ ]:
def handle_outliers(X_train, Y_train, X_test):

  ND_features = ['temp', 'humidity', 'log_pollution']
  NND_features = ['windspeed']

  X_train, bounds = handle_ND_outliers_train(X_train, ND_features)
  X_test = handle_ND_outliers_test(X_test, bounds)

  contamination = 0.001
  threshold = 0.001
  X_train, Y_train = handle_NND_outliers_train(X_train, Y_train, threshold, contamination, NND_features)

  return X_train, Y_train, X_test

Before applying the changes, let's see the inital state (Just for the purposes of visualization we will use the data of the tree models only: X_train_tree & X_val_tree because in any case the features of cleaning the outliers are the same for the trees and lr models):

In [ ]:
X_train_tree.describe()

In [ ]:
X_val_tree.describe()

In [ ]:
# selected_features = ['windspeed', 'log_pollution']
df = X_train_tree[NND_features]
df.hist()
# Add x and y labels
plt.xlabel("Windspeed", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.title("Histogram of NND Features", fontsize=14)
plt.show()

Let's see what outliers will be removed in the normally distributed features: (using the plotting function we created in the visualizations stage)

In [ ]:
IQR_train = pd.concat([X_train_tree, Y_train_tree], axis=1)
# IQR_X_train = X_train[['temp', 'humidity', 'log_pollution']]
plot_normal_dist_outliers(IQR_train, ND_features)

Implementing the changes:

In [ ]:
X_train_tree, Y_train_tree, X_test_tree = handle_outliers(X_train_tree, Y_train_tree, X_val_lr)
X_train_lr, Y_train_lr, X_test_lr = handle_outliers(X_train_lr, Y_train_lr, X_val_lr)

Now, we'll assess the impact

In [ ]:
plot_normal_dist_outliers(pd.concat([X_train_tree, Y_train_tree], axis=1), ND_features)

We can observe that the outliers in the normally distributed features were reduced drastically!

And the other non-distributed features:

In [ ]:
df = X_train_tree[NND_features]
df.hist()
# Add x and y labels
plt.xlabel("Windspeed", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.title("Histogram of NND Features", fontsize=14)
plt.show()

In [ ]:
X_train_tree.describe()

In [ ]:
X_val_tree.describe()

We can conclude that indeed the outlier number has been dropped and the changes expected were made.

## Data Normalizing

this is only relevant to lr model.

In [ ]:
def fit_normalize_data(df, scaler, col):
    df[[col]] = scaler.fit_transform(df[[col]])
    return df, scaler

def transform_normalize_data(df, scaler, col):
    df[[col]] = scaler.transform(df[[col]])
    return df

In [ ]:
def normalize_data(X_train, X_val):
  columns_to_normalize = ['temp', 'humidity', 'windspeed','log_pollution']
  scaler = StandardScaler()
  for col in columns_to_normalize:
    X_train, scaler = fit_normalize_data(X_train, scaler, col)
    X_val = transform_normalize_data(X_val, scaler, col)

  return X_train, X_val

In [ ]:
X_train_lr, X_val_lr = normalize_data(X_train_lr, X_val_lr)

In [ ]:
X_train_lr

In [ ]:
X_val_lr

## PCA - Dimensionality Reduction

Before performing the PCA let's look at the Cumulativ Explained Variance Ratio plot:

In [ ]:
variance_vals = [0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 0.995, 0.9999]
var_dim = []

for variance in variance_vals:
    pca = PCA(variance)
    data = pca.fit_transform(X_train)
    print("For explained variance:", variance, "number of dimensions:", data.shape[1])
    var_dim.append(data.shape[1])

plt.plot(variance_vals, var_dim)
plt.xlabel("Explained Variance")
plt.ylabel("Number of Dimensions")
plt.title("Number of Dimensions per Explained Variance")
plt.show()


We can evaluate that most of the variance is already gained after approximately 12 dimensions. Therefore, We will choose: threshold=0.95.

In [ ]:
def perform_pca_with_variance(variance_threshold, train_data):
    pca = PCA(variance_threshold)
    transformed_data = pca.fit_transform(train_data)
    return pca, transformed_data

def apply_pca_transformation(pca, data):
    transformed_data = pca.transform(data)
    return transformed_data

In [ ]:
variance_threshold = 0.95

pca, X_train_lr_reduced = perform_pca_with_variance(variance_threshold, X_train_lr)
X_train_lr_reduced = pd.DataFrame(X_train_lr_reduced)

X_val_lr_reduced = apply_pca_transformation(pca, X_val_lr)
X_val_lr_reduced = pd.DataFrame(X_val_lr_reduced)

In [ ]:
X_train_lr_reduced

In [ ]:
X_val_lr_reduced

## Final Preprocessing Function

Now, after deciding what manipulations we will do on the train data (Outliers removal, Normalization, Missing values handling, Categorial data handling, Feature selection and Feature manipulation), we will create a generic preprocess function to run it all at once.

In [ ]:
def preprocess_data_for_lr(X_train, Y_train, X_test):

  # Add & remove features - 'proportion_imports'
  X_train = feature_engineering(X_train)
  X_test = feature_engineering(X_test)

  # Categorical Data Handling
  X_train = handle_categorial_data(X_train)
  X_test = handle_categorial_data(X_test)

  # Outliers Removal
  X_train, Y_train, X_test = handle_outliers(X_train, Y_train, X_test)

  # Normalization
  X_train, X_test = normalize_data(X_train, X_test)

  # PCA
  pca, X_train = perform_pca_with_variance(variance_threshold, X_train)
  X_train = pd.DataFrame(X_train)

  X_test = apply_pca_transformation(pca, X_test)
  X_test = pd.DataFrame(X_test)

  return X_train, Y_train, X_test

In [ ]:
def preprocess_data_for_tree(X_train, Y_train, X_test):

  # Add & remove features - 'proportion_imports'
  X_train = feature_engineering(X_train)
  X_test = feature_engineering(X_test)

  # Outliers Removal
  X_train, Y_train, X_test = handle_outliers(X_train, Y_train, X_test)

  return X_train, Y_train, X_test

In [ ]:
X_train_lr_ppc, Y_train_lr_ppc, X_val_lr_ppc = preprocess_data_for_lr(X_train_lr_original, Y_train_lr_original, X_val_lr_original)

In [ ]:
X_train_lr_ppc

In [ ]:
X_val_lr_ppc

In [ ]:
X_train_tree_ppc, Y_train_tree_ppc, X_val_tree_ppc = preprocess_data_for_tree(X_train_tree_original, Y_train_tree_original, X_val_tree_original)

In [ ]:
X_train_tree_ppc

In [ ]:
X_val_tree_ppc

# TODO - NEED TO DEL!!!

In [ ]:
# # TODO Added : change accordingly
# def preprocess_data_v2(X_train, Y_train, X_test):

#   # Add & remove features - 'proportion_imports'
#   X_train = feature_engineering_v2(X_train)
#   X_test = feature_engineering_v2(X_test)

#   # Categorical Data Handling
#   # X_train = handle_categorial_data(X_train)
#   # X_test = handle_categorial_data(X_test)

#   # Outliers Removal
#   # X_train, Y_train, X_test = handle_outliers(X_train, Y_train, X_test)

#   # Normalization
#   # X_train, X_test = normalize_data(X_train, X_test)

#   return X_train, Y_train, X_test

In [ ]:
# def preproccess_data_with_pca(X_train, Y_train, X_test):

#   X_train, Y_train, X_test = preprocess_data(X_train, Y_train, X_test)

#   # PCA - Dimensionality Reduction
#   pca, X_train = perform_pca_with_variance(variance_threshold, X_train)
#   X_train = pd.DataFrame(X_train)

#   X_test = apply_pca_transformation(pca, X_test)
#   X_test = pd.DataFrame(X_test)

#   return X_train, Y_train, X_test

In [ ]:
# # Applying all the preprocessing decisions at once from start to end
# X_train_ppc, Y_train_ppc, X_val_ppc = preprocess_data(X_train_original, Y_train_original, X_val_original)
# # Y_val_original

In [ ]:
# X_train_pca_ppc, Y_train_pca_ppc, X_val_pca_ppc = preproccess_data_with_pca(X_train_original, Y_train_original, X_val_original)

In [ ]:
# X_train_ppc

In [ ]:
# X_val_ppc

In [ ]:
# X_train_pca_ppc

In [ ]:
# X_val_pca_ppc

# **Modelling**

A dictionary that consists the 3 algorithms we will use ...

In [ ]:
# # TODO : this is the working one
# X_train_ppc, Y_train_ppc, X_val_ppc = preprocess_data_v2(X_train_original, Y_train_original, X_val_original)

In [ ]:
# X_train_ppc

In [ ]:
models = {'Random forest': None,
          'Linear regression': None,
          'XGBOOST' : None}

We chose to use grid search in order to find the best parameters for each model and the parameters that are not mentioned in the Grid search, we took their defult values.

## Random forest

In [ ]:
def run_random_forest(x_train, y_train):
  rf = RandomForestRegressor(n_estimators=1000, random_state=42)
  tuned_rf = GridSearchCV(estimator=rf,
                          param_grid={
                              'max_features': ['auto', 'sqrt', 'log2', 1/3],
                              'max_depth': [None, 10,30],
                              'min_samples_leaf': [1, 2, 4]},
                          scoring='neg_root_mean_squared_error',
                          cv=3,
                          verbose=3,
                          refit=True)
  tuned_rf.fit(x_train, y_train)
  return tuned_rf

In [ ]:
#Random Forest
#TODO - OLD ONE - TO CHECK IF NEEDED
# df = X_train.copy()
# valid_df = X_val.copy()
# df_labels = Y_train.copy()
# valid__df_labels= Y_val.copy()

# # Converting 'datetime' Feature
# df['datetime'] = pd.to_datetime(df['datetime'])
# df['year'] = df['datetime'].dt.year
# df['month'] = df['datetime'].dt.month
# df['day'] = df['datetime'].dt.day
# df['hour'] = df['datetime'].dt.hour

# # Drop the original 'datetime' column if needed
# df = df.drop('datetime', axis=1)

# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import GridSearchCV, train_test_split
# from sklearn.metrics import mean_squared_error

#rf_model = RandomForestRegressor(random_state=42)
#param_grid = {
#    'n_estimators': [100, 1000],
#    'max_depth': [None, 10,30],
#    'min_samples_split': [2, 5],
#    'min_samples_leaf': [1, 2, 4]
#}

#grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
#grid_search.fit(df, df_labels)

#best_params = grid_search.best_params_
#print("Best Hyperparameters:", best_params)

#Evaluation
#best_rf_model = grid_search.best_estimator_
#y_pred = best_rf_model.predict(valid_df)

#mse = mean_squared_error(valid__df_labels, y_pred)
#rmse = mse**0.5
#print("Root Mean Squared Error on Test Set:", rmse)



## Linear Regression

In [ ]:
def run_linear_regression(x_train, y_train):
  lr = LinearRegression()
  lr.fit(x_train, np.ravel(y_train))
  cv_score = cross_val_score(lr, x_train, y_train, scoring='neg_root_mean_squared_error', cv=3)
  return lr, cv_score

In [ ]:

# #Linear Regression Model
# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import cross_val_score

# def lr_model(x_train, y_train):
#   lr = LinearRegression()
#   lr.fit(x_train,y_train)
#   cv_score = cross_val_score(lr, x_train, y_train, scoring='neg_mean_squared_error', cv=3)

#   # Convert scores to positive values and calculate the mean
#   rmse_score = (-cv_score)**0.5
#   mean_rmse = rmse_score.mean()

#   print("Cross-Validation RMSE Scores:", rmse_score)
#   print("Mean Cross-Validation RMSE:", mean_rmse)
#   return lr, cv_score


## XGBoost

In [ ]:
#XGboost
def run_xgboost(x_train,y_train):
  xgb_model = xgb.XGBRegressor(objective='reg:squarederror',n_estimators=1000,random_state=42)
  tuned_xgb = GridSearchCV(estimator=xgb_model,
                          param_grid={'max_features': ['auto','log2', 1/3],
                                      'max_depth' : [3,5,7]},
                          scoring='neg_root_mean_squared_error',
                          cv=3,
                          verbose=3,
                          refit=True)
  tuned_xgb.fit(x_train, y_train)
  return tuned_xgb

In [ ]:
#possible prameters for xgboost
#param_grid = {
#    'learning_rate': [0.01, 0.1, 0.2],
#    'n_estimators': [50, 100, 200],
#    'max_depth': [3, 5, 7],
#    'min_child_weight': [1, 3, 5],
#    'subsample': [0.7, 0.8, 0.9],
#    'gamma': [0, 0.1, 0.2],
#    'colsample_bytree': [0.7, 0.8, 0.9],
#    'alpha': [0, 0.1, 0.5],
#    'lambda': [0, 0.1, 0.5]
#}

## Run all models

Let's run all models:

In [ ]:
def run_tree_models(x, y):
  models_dict = {}
  models_dict['Random forest'] = run_random_forest(x, y)
  models_dict['XGBOOST'] = run_xgboost(x,y)
  return models_dict

In [ ]:
models = run_all_models(X_train_tree_ppc, Y_train_tree_ppc.values.ravel())

In [ ]:
models['Linear regression'] = run_linear_regression(_train_lr_ppc, Y_train_lr_ppc)

In [ ]:
models

# TODO - NEED TO DEL!

In [ ]:
# def run_all_models(x, y):
#   models_dict = {}
#   models_dict['Random forest'] = run_random_forest(x, y)
#   models_dict['Linear regression'] = run_linear_regression(x, y)
#   models_dict['XGBOOST'] = run_xgboost(x,y)
#   return models_dict

In [ ]:
# X_train_ppc

In [ ]:
# models = run_all_models(X_train_ppc, Y_train_ppc.values.ravel())

In [ ]:
# models

Comparing their cross validation scores:

In [ ]:
cv_scores = {}
cv_scores['Random forest'] = models['Random forest'].best_score_
cv_scores['Linear regression'] = models['Linear regression'][1].mean()
cv_scores['XGBOOST'] = models['XGBOOST'].best_score_

cv_scores

In [ ]:
#TODO CHOOSE IF RELEVNT , IF IT DOES - EXPLAIN IT
sns.set()


cv_scores_df = pd.DataFrame.from_dict(cv_scores, orient='index')
cv_scores_df.plot.bar(rot=45, legend=False)

In [ ]:
cv_scores

Best  Model Parameters

In [ ]:
#Finding the best Parameters
# parameters found by grid search for RF and XGboost models

print("Random Forests best parameters are :", models['Random forest'].best_params_)
print("xgboost best parameters are:" ,models['XGBOOST'].best_params_)


# Evaluating on validation set

In [ ]:
def evaluate_single_model(x_test, y_test, model):
  y_pred = model.predict(x_test)
  return np.sqrt(mean_squared_error(y_test, y_pred))

And test the function:

In [ ]:
evaluate_single_model(X_val_tree_ppc, Y_val_tree_original, models['Random forest'].best_estimator_)

Let's create a dictionary that includes only trained models:

In [ ]:
trained_models_dict = {}
trained_models_dict['Random forest'] = models['Random forest'].best_estimator_
trained_models_dict['Linear regression'] = models['Linear regression'][0]
trained_models_dict['XGBOOST'] = models['XGBOOST'].best_estimator_
trained_models_dict

And create a function that iterates over all models: (We added a condition of a PCA that we may do later, Only on the Linear Regression).

In [ ]:
def evaluate_all_models(x, y, x_lr, y_lr, models_dict):
  test_set_scores = {}
  for k, v in models_dict.items():
    if k == 'Linear regression':
      test_set_scores[k] = evaluate_single_model(x_lr, y, v)
    else:
      test_set_scores[k] = evaluate_single_model(x, y, v)
  return test_set_scores

Finally, let's run our function:

In [ ]:
test_set_scores = evaluate_all_models(X_val_tree_ppc, Y_val_tree_original, X_val_lr_ppc, Y_val_lr_original, trained_models_dict)
test_set_scores

### Overall comparison

Let's combine the two dictionaries:

In [ ]:
combined_dict = {k: [np.abs(v), test_set_scores[k]] for k, v in cv_scores.items()}
combined_dict

And compare the CV score to the test set score:

In [ ]:
scores_df = pd.DataFrame.from_dict(combined_dict, orient='index', columns=['CV score', 'Test set score'])
scores_df

## Feature selection & importance

Here we can learn and understand which of the models are more or less important and we can delete, add or change any features accordingly.

Let's find the importance of all features for each model and remove insignificant features:

In [ ]:
feature_importance_dict = {}

#### Tree based methods

For tree based models we can use SKLearn's built-in methods:

In [ ]:
def find_tree_feature_importance(model, columns):
  importance = model.feature_importances_
  importance *= 100 / np.max(importance)  # Normalize
  importance = pd.DataFrame(importance, index=columns, columns=["Importance"])
  importance = importance.sort_values(by=['Importance'], ascending=False)
  return importance

In [ ]:
feature_importance_dict['Random forest'] = find_tree_feature_importance(trained_models_dict['Random forest'], X_train_tree_ppc.columns)
feature_importance_dict['XGBOOST'] = find_tree_feature_importance(trained_models_dict['XGBOOST'], X_train_tree_ppc.columns)

In [ ]:
feature_importance_dict

#### Classical regression methods

One interpetation of feature importance for linear/ridge regression, is the normalized value of the estimator's coefficients:

In [ ]:
def find_normalized_lr_feature_importance(model, x_train):
  coefficients = {x_train.columns[i]: np.abs(model.coef_[i]) for i in range(len(x_train.columns))}
  coefficients_df = pd.DataFrame.from_dict(coefficients, orient='index', columns=['Importance'])
  coefficients_df['Importance'] *=  x_train.std()
  coefficients_df['Importance'] *= 100 / coefficients_df['Importance'].max()
  importance = coefficients_df.sort_values(by=['Importance'], ascending=False)
  return importance

In [ ]:
feature_importance_dict['Linear regression'] = find_normalized_lr_feature_importance(trained_models_dict['Linear regression'], X_train_lr_ppc)

#### Comparison

Let's visualize the different feature imprtance across the models:

In [ ]:
sns.set()

fig, axes = plt.subplots(2, 2, figsize=(12, 12))
feature_importance_dict['Random forest'].plot(kind='barh', ax=axes[0, 0], legend=False, title='Random forest')
feature_importance_dict['XGBOOST'].plot(kind='barh', ax=axes[0, 1], legend=False, title='XGBoost')
feature_importance_dict['Linear regression'].plot(kind='barh', ax=axes[1, 0], legend=False, title='Linear regression')

## Add/Remove Features to Improve Results

Lets try to improve our data to get better results (we will use the original dfs and then run the Preproccessing function):
 - We will remove features according to feature importance methods (comparison)


Removing weak features:

* Let's find the 7 weakest features in each model:

In [ ]:
weak_features_dict = {}
for k, v in feature_importance_dict.items():
  weak_features_dict[k] = list(v.index.values[-7:])
weak_features_dict

And remove these features from each model. We create a dictionary mapping each model to X_train_ppc and X_test_ppc without the last 3 features:

In [ ]:
def remove_weakest_features(X_train, X_test):
  x_data_dict = {}
  for k, v in weak_features_dict.items():
    x_data_dict[k] = (X_train.drop(v, axis=1), X_test.drop(v, axis=1))

  return X_train, X_test

In [ ]:
X_train_tree_ppc, X_val_tree_ppc = remove_weakest_features(X_train_tree_ppc, X_val_tree_ppc )
X_train_lr_ppc, X_val_lr_ppc = remove_weakest_features(X_train_lr_ppc, X_val_lr_ppc)

## Model improvement

Running the models again after the changes:

In [ ]:
models = run_tree_models(X_train_tree_ppc, Y_train_tree_ppc.values.ravel())

In [ ]:
models['Linear regression'] = run_linear_regression(_train_lr_ppc, Y_train_lr_ppc)

In [ ]:
models

#TODO - NEED TO DEL!!!

* We saw that we have a lot of features, so we will decide to perform PCA for the **linear regression**

In [ ]:
# models['Linear regression PCA'] = run_linear_regression(X_train_pca_ppc, Y_train_pca_ppc)

In [ ]:
models

In [ ]:
cv_scores = {}
cv_scores['Random forest'] = models['Random forest'].best_score_
cv_scores['Linear regression PCA'] = models['Linear regression PCA'][1].mean()
cv_scores['Linear regression'] = models['Linear regression'][1].mean()
cv_scores['XGBOOST'] = models['XGBOOST'].best_score_

In [ ]:
trained_models_dict = {}
trained_models_dict['Random forest'] = models['Random forest'].best_estimator_
trained_models_dict['Linear regression'] = models['Linear regression'][0]
trained_models_dict['Linear regression PCA'] = models['Linear regression PCA'][0]
trained_models_dict['XGBOOST'] = models['XGBOOST'].best_estimator_
trained_models_dict

In [ ]:
# test_set_scores = evaluate_all_models(X_val_ppc, Y_val_original, X_val_pca_ppc, trained_models_dict)
# test_set_scores

In [ ]:
# test_set_scores = evaluate_all_models(X_val_ppc, Y_val_original, X_val_pca_ppc, trained_models_dict)
# test_set_scores

In [ ]:
test_set_scores = evaluate_all_models(X_val_tree_ppc, Y_val_tree_original, X_val_lr_ppc, Y_val_lr_original, trained_models_dict)
test_set_scores

In [ ]:
combined_dict = {k: [np.abs(v), test_set_scores[k]] for k, v in cv_scores.items()}
combined_dict

Let's look at the new:

In [ ]:
new_scores_df = pd.DataFrame.from_dict(combined_dict, orient='index', columns=['CV score removed features', 'Test set score removed features'])
new_scores_df

And combine to one df:

In [ ]:
combined_df = pd.concat([scores_df, new_scores_df], axis=1, ignore_index=False)
combined_df

Finally, let's plot the results:



In [ ]:
combined_df[['CV score', 'CV score removed features']].plot(kind='barh', title = 'CV Comparison').legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

In [ ]:
combined_df[['Test set score', 'Test set score removed features']].plot(kind='barh', title = 'Test set score Comparison').legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

# Prediction - Runing on Test

We saw that ___ is the best model, this is a tree based model then we will do the preprocessing according to that.

Copy of the test:

In [ ]:
X_test_original = test_data.copy()

PPC on the test:

In [ ]:
X_test_original = dealing_with_times(X_test_original)

In [ ]:
# Applying all the preprocessing decisions at once from start to end
X_train_tree_ppc, Y_train_tree_ppc, X_test_tree_ppc = preprocess_data(X_train_tree_original, Y_train_tree_original, X_test_original)

In [ ]:
# X_train_pca_ppc, Y_train_pca_ppc, X_test_pca_ppc = preproccess_data_with_pca(X_train_original, Y_train_original, X_test_original)

Running the best model to find test predictions:

In [ ]:
#Our predictions table: (of our best model XGBoost)
y_pred = models['XGBOOST'].best_estimator_.predict(X_test_tree_ppc)
y_pred = pd.DataFrame(y_pred,columns = ['Predictions'])
y_pred

# **Output**

צריך למלא את זה לפי מה שיצא לנו!!!


In [ ]:
# Keep keys the same, and replace values according to your results and the specified type

results = {'model': ['string1', 'string2', 'string3'],
           'Score (RMSE)': ['string1', 'string2', 'string3'],
           'Hyperparams used': [['list1'], ['list2'], ['list3']],
           'Features dropped': [['list1'], ['list2'], ['list3']],
           'New features created': [['list1'], ['list2'], ['list3']],
           'Runtime trainining + inference (seconds)': ['int1', 'int2', 'int3'],
           'Hardware used (GPU/CPU/TPU)': ['string1', 'string2', 'string3'],
           'Explainability (top 3 features)': [['list1'], ['list2'], ['list3']]
           }

results = pd.DataFrame(results)
results

In [ ]:
#Maybe to DEL
#results.to_csv(os.path.join(os.getcwd(), 'resultsEx1.csv'))

In [ ]:
# Export to csv test predictions to the git
new_url = 'https://raw.githubusercontent.com/ariel-hedvat/AdvancedMLDLCourseAssignments/main/Assignment%20I/resultsEx1.csv'
results.to_csv(new_url, index=False)
print(f'The new CSV file has been exported to: {new_url}')